# Dangerous driver detection

In this exercise, we will classify images of drivers and decide if they exhibit any of one of nine dangerous behaviours, or whether they are deemed to drive safely.

## Setup

In [1]:
%matplotlib inline

Set the path to input data. Change this to run on some other box.

In [2]:
inputdir = '/home/ubuntu/data/state-farm'
model_path = "http://files.fast.ai/models/"

A bunch of generic imports.

In [3]:
import os
import numpy as np
np.set_printoptions(precision=4, linewidth=100)

Load (and reload) the utility code we use in later steps.

In [5]:
from imp import reload
import utils; reload(utils)

<module 'utils' from 'utils.pyc'>

Keras setup

In [6]:
from numpy.random import random, permutation

import keras
from keras import backend as K
from keras.utils.data_utils import get_file
from keras.models import Sequential, Model
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.layers import Input
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import Adam
from keras.preprocessing import image

# SPECIAL GLOBAL MAGIC SETTING THAT IF WE DON'T SET IT MAKES THINGS BLOW UP!!!  :-/
K.set_image_dim_ordering('th')

Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

WARNING (theano.sandbox.cuda): CUDA is installed, but device gpu is not available  (error: Unable to get the number of gpus available: unknown error)


In [7]:
## Model creation

We'll try building a Vgg16 model and fine tune it, to see how far that gets us.

In [8]:
def add_conv_block(model, layers, filters):
    for i in range(layers): 
        model.add(ZeroPadding2D((1, 1)))
        model.add(Convolution2D(filters, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

def add_fully_connected_block(model):
    model.add(Dense(4096, activation = 'relu'))
    model.add(Dropout(0.5))

# Mean of each channel as provided by VGG researchers
vgg_mean = np.array([123.68, 116.779, 103.939]).reshape((3, 1, 1))

def vgg_preprocess(x):
    x = x - vgg_mean     # subtract mean
    return x[:, ::-1]    # reverse axis bgr->rgb

def vgg16_model():
    model = Sequential()
    model.add(Lambda(vgg_preprocess, input_shape = (3, 224, 224)))
    
    add_conv_block(model, 2, 64)
    add_conv_block(model, 2, 128)
    add_conv_block(model, 3, 256)
    add_conv_block(model, 3, 512)
    add_conv_block(model, 3, 512)

    model.add(Flatten())
    add_fully_connected_block(model)
    add_fully_connected_block(model)
    model.add(Dense(1000, activation='softmax'))
    return model

Create a model and load weights into it.

In [10]:
model = vgg16_model()
fpath = get_file('vgg16.h5', model_path + 'vgg16.h5', cache_subdir='models')
#model.load_weights(fpath)

/home/ubuntu/anaconda2/envs/fastai/lib/python2.7/site-packages/keras/layers/core.py:622: UserWarning: `output_shape` argument not specified for layer lambda_2 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 3, 224, 224)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


MemoryError: 